# Exámenes A1 y A3 de APR en el grupo 4CO11, turno 1

# Examen A1: ejercicio con Fashion-MNIST

Los ejercicios de las sesiones de laboratorio del bloque 1 se han basado en el corpus Fashion-MNIST. Hemos utilizado el código siguiente para leer Fashion-MNIST con su partición train-test estándar, normalizar las imágenes a $\,[0,1]\,$, y establecer una partición train-val-test mediante partición del train estándar en train-val.

In [7]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'; import keras
input_dim = 784; num_classes = 10
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_val = x_train_val.reshape(-1, input_dim).astype("float32") / 255.0
x_test = x_test.reshape(-1, input_dim).astype("float32") / 255.0
y_train_val = keras.utils.to_categorical(y_train_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]
print(f'train {x_train.shape} {y_train.shape} val {x_val.shape} {y_val.shape} test {x_test.shape} {y_test.shape}')

train (50000, 784) (50000, 10) val (10000, 784) (10000, 10) test (10000, 784) (10000, 10)


La mejor precisión en test encontrada, $\,89.6\%,\,$ se ha hallado con los siguientes hiperparámetros: arquitectura de MLP con una capa oculta de $\,800\,$ RELUs; optimizador Adam con learning rate $\,0.00015\,$ y batch size $\,256;\;$ planificador ReduceLROnPlateau con factor $\,0.32\,$ y paciencia $\,5;\;$ y regularización mediante early stopping con paciencia $\,10.$ A continuación se define una función que, dados inicializadores de kernel y bias, realiza un experimento con dichos hiperparámetros y devuelve la precisión en test encontrada:

In [8]:
def run_exp(kernel_initializer="glorot_uniform", bias_initializer="zeros"):
  M = keras.Sequential()
  M.add(keras.Input(shape=(784,)))
  M.add(keras.layers.Dense(units=800, activation='relu',
    kernel_initializer=kernel_initializer, bias_initializer=bias_initializer))
  M.add(keras.layers.Dense(10, activation='softmax',
    kernel_initializer=kernel_initializer, bias_initializer=bias_initializer))
  opt = keras.optimizers.Adam(learning_rate=0.00015)
  M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
  reduce_cb = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.32, patience=5, restore_best_weights=True)
  early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, min_delta=1e-5)
  M.fit(x_train, y_train, batch_size=256, epochs=100, verbose=0, validation_data=(x_val, y_val),
    callbacks=[reduce_cb, early_cb])
  _, acc = M.evaluate(x_test, y_test, verbose=0)
  return acc

Los inicializadores de kernel y bias por omisión, GlorotUniform y ceros, son los mismos que emplea keras por omisión. Dado que GlorotUniform es una inicialización aleatoria, cada vez que realicemos el experimento anterior obtendremos una precisión en test (ligeramente) distinta. Veamos la precisión en test que se obtiene en un experimento aislado con una semilla aleatoria $23$; debería ser próxima a $\,89.6\%,\,$ si bien puede variar a causa del indeterminismo intrínseco del cálculo masivamente paralelo y, en general, de diferencias en el entorno de ejecución.

In [3]:
import time; start = time.time()
keras.utils.set_random_seed(seed=23); acc = run_exp(); print(f'Precisión: {acc:.2%}')
print('Tiempo (hh:mm:ss):', time.strftime('%H:%M:%S', time.gmtime(time.time() - start)))

I0000 00:00:1733933806.457772   38295 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2616 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1733933808.028319   39942 service.cc:148] XLA service 0x7fac580046e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733933808.028353   39942 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
I0000 00:00:1733933808.127747   39942 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1733933808.853674   39942 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Precisión: 89.26%
Tiempo (hh:mm:ss): 00:00:28


Conviene repetir el experimento varias veces para hallar una estimación fiable de la precisión en test que cabe esperar:

In [9]:
import time; start = time.time()
keras.utils.set_random_seed(seed=23); num_exp = 10; acc = np.zeros(num_exp)
for exp in range(num_exp):
    acc[exp] = run_exp()
print(f'Precisión media: {acc.mean():.2%}  Desviación estándar: {acc.std():.2%}')
print('Tiempo (hh:mm:ss):', time.strftime('%H:%M:%S', time.gmtime(time.time() - start)))

I0000 00:00:1733934655.719036  149332 service.cc:148] XLA service 0x7f82d80081c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733934655.719060  149332 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
I0000 00:00:1733934655.811875  149332 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1733934656.469199  149332 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Precisión media: 89.52%  Desviación estándar: 0.09%
Tiempo (hh:mm:ss): 00:04:54


**Ejercicio (2 puntos):** $\;$ escoge un inicializador de kernel aleatorio y distinto a GlorotUniform, y estima la precisión en test mediante repetición del experimento anterior dos veces al menos.

In [ ]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'; import keras
input_dim = 784; num_classes = 10
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train_val = x_train_val.reshape(-1, input_dim).astype("float32") / 255.0
x_test = x_test.reshape(-1, input_dim).astype("float32") / 255.0
y_train_val = keras.utils.to_categorical(y_train_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]
print(f'train {x_train.shape} {y_train.shape} val {x_val.shape} {y_val.shape} test {x_test.shape} {y_test.shape}')

train (50000, 784) (50000, 10) val (10000, 784) (10000, 10) test (10000, 784) (10000, 10)


In [1]:
# CAMBIAMOS glorot_uniform por otro kernel_initializer aleatorio, por e
def run_exp(kernel_initializer="random_normal", bias_initializer="zeros"):
  M = keras.Sequential()
  M.add(keras.Input(shape=(784,)))
  M.add(keras.layers.Dense(units=800, activation='relu',
    kernel_initializer=kernel_initializer, bias_initializer=bias_initializer))
  M.add(keras.layers.Dense(10, activation='softmax',
    kernel_initializer=kernel_initializer, bias_initializer=bias_initializer))
  opt = keras.optimizers.Adam(learning_rate=0.00015)
  M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
  reduce_cb = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.32, patience=5, restore_best_weights=True)
  early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, min_delta=1e-5)
  M.fit(x_train, y_train, batch_size=256, epochs=100, verbose=0, validation_data=(x_val, y_val),
    callbacks=[reduce_cb, early_cb])
  _, acc = M.evaluate(x_test, y_test, verbose=0)
  return acc

In [10]:
import time; start = time.time()
keras.utils.set_random_seed(seed=23); num_exp = 3; acc = np.zeros(num_exp)
for exp in range(num_exp):
    acc[exp] = run_exp()
print(f'Precisión media: {acc.mean():.2%}  Desviación estándar: {acc.std():.2%}')
print('Tiempo (hh:mm:ss):', time.strftime('%H:%M:%S', time.gmtime(time.time() - start)))

Precisión media: 89.50%  Desviación estándar: 0.07%
Tiempo (hh:mm:ss): 00:01:24


# Examen A3: ejercicio con CIFAR-10

En las dos primeras sesiones de laboratorio del bloque 2 seguimos con MNIST y Fashion-MNIST; vimos que con CNNs sencillas convenientemente regularizadas se obtenían precisiones en test muy buenas, del $\,99.5\%\,$ en MNIST y $\,92.0\%\,$ en Fashion-MNIST. A partir de la tercera sesión de laboratorio del bloque 2 utilizamos el corpus de imágenes a color CIFAR-10. El siguiente código lee CIFAR-10 con su partición train-test estándar y establece una partición train-val-test mediante partición del train estándar en train-val.

In [11]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras; from keras import layers
keras.utils.set_random_seed(23)
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train_val = x_train_val.astype("float32")
x_test = x_test.astype("float32")
y_train_val = keras.utils.to_categorical(y_train_val, 10)
y_test = keras.utils.to_categorical(y_test, 10)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

(40000, 32, 32, 3) (40000, 10) (10000, 32, 32, 3) (10000, 10) (10000, 32, 32, 3) (10000, 10)


En la tercera sesión del bloque 2 (sesión 8) vimos que una CNN sencilla obtenía precisiones alrededor del $73\%$. A continuación se define una función para realizar un experimento con dicha CNN sencilla. La normalización de imágenes, a $\,[-1, 1],\,$ se integra como primera capa tras la de entrada. Tras la normalización, la red aplica dos capas convolucionales de 32 y 64 filtros $\,3\times 3$. Cada capa convolucional viene seguida por una capa de agrupación MaxPooling con ventana $2\times 2.$ Tras los dos pares Conv2D-MaxPooling2D, la salida se aplana y se procesa mediante un MLP como el del examen A1, si bien en este caso se añade una capa de regularización Dropout con probabilidad $\,0.5\,$ tras la capa oculta de $\,800\,$ unidades.

In [12]:
def run_exp2():
  M = keras.Sequential()
  M.add(keras.Input(shape=(32, 32, 3)))
  M.add(layers.Rescaling(scale=1 / 127.5, offset=-1))
  M.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
  M.add(layers.MaxPooling2D(pool_size=(2, 2)))
  M.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu"))
  M.add(layers.MaxPooling2D(pool_size=(2, 2)))
  M.add(layers.Flatten())
  M.add(layers.Dense(units=800, activation='relu'))
  M.add(layers.Dropout(0.5))
  M.add(layers.Dense(10, activation='softmax'))
  opt = keras.optimizers.Adam(learning_rate=0.00015)
  M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
  reduce_cb = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.32, patience=5, restore_best_weights=True)
  early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, min_delta=1e-5)
  M.fit(x_train, y_train, batch_size=256, epochs=100, verbose=1, validation_data=(x_val, y_val),
    callbacks=[reduce_cb, early_cb])
  _, acc = M.evaluate(x_test, y_test, verbose=0)
  return acc

Repetimos `run_exp2` para comprobar que la CNN sencilla obtiene una precisión próxima al $73\%$:

In [14]:
import time; start = time.time()
keras.utils.set_random_seed(seed=23); num_exp = 3; acc = np.zeros(num_exp)
for exp in range(num_exp):
    acc[exp] = run_exp2()
print(f'Precisión media: {acc.mean():.2%} Desviación estándar: {acc.std():.2%}')
print('Tiempo (hh:mm:ss):', time.strftime('%H:%M:%S', time.gmtime(time.time() - start)))

Precisión media: 73.02% Desviación estándar: 0.13%
Tiempo (hh:mm:ss): 00:06:43


Tras realizar un experimento similar al anterior, en las sesiones tercera y cuarta del bloque 2 nos centramos en el uso de redes pre-entrenadas con el fin de obtener precisiones por encima del $\,90\%\,$ en CIFAR-10. Concretamente, hicimos uso de una ResNet50V2 descabezada para aprender, mediante transfer learning y fine-tuning, redes que superaban el $\,95\%$. Además, en línea con tendencias populares de los últimos años, conseguimos aumentar ligeramente las precisiones alcanzadas haciendo uso de aumento de datos. Por simplicidad, en este examen no haremos uso de redes pre-entrenadas, pero sí consideraremos la posibilidad de mejorar una CNN sencilla con aumento de datos.

**Ejercicio (2 puntos):** $\;$ Define una nueva función `run_exp3` para experimentar con la CNN sencilla y aumento de datos. La API de las capas básicas de aumento de imágenes está en [https://keras.io/api/layers/preprocessing_layers](https://keras.io/api/layers/preprocessing_layers). En las sesiones de laboratorio ya hemos utilizado `RandomFlip` y `RandomTranslation`. Escoge las capas de aumento de imágenes que consideres prometedoras y aplícalas adecuadamente. Repite `run_exp3` al menos dos veces para estimar la precisión de la CNN sencilla con el aumento de datos definido; deberías obtener un $75\%$ al menos. 

In [2]:
import numpy as np; import matplotlib.pyplot as plt
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras; from keras import layers
keras.utils.set_random_seed(23)
(x_train_val, y_train_val), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train_val = x_train_val.astype("float32")
x_test = x_test.astype("float32")
y_train_val = keras.utils.to_categorical(y_train_val, 10)
y_test = keras.utils.to_categorical(y_test, 10)
x_train = x_train_val[:-10000]; x_val = x_train_val[-10000:]
y_train = y_train_val[:-10000]; y_val = y_train_val[-10000:]
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

2024-12-11 18:49:18.403303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733939358.457242  854554 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733939358.471055  854554 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(40000, 32, 32, 3) (40000, 10) (10000, 32, 32, 3) (10000, 10) (10000, 32, 32, 3) (10000, 10)


In [3]:
def run_exp3():
  M = keras.Sequential()
  M.add(keras.Input(shape=(32, 32, 3)))
  M.add(layers.Rescaling(scale=1 / 127.5, offset=-1))
  M.add(layers.RandomFlip("horizontal"))
  # M.add(layers.RandomTranslation(0.2, 0.2, fill_mode="nearest"))
  M.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
  M.add(layers.MaxPooling2D(pool_size=(2, 2)))
  M.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu"))
  M.add(layers.MaxPooling2D(pool_size=(2, 2)))
  M.add(layers.Flatten())
  M.add(layers.Dense(units=800, activation='relu'))
  M.add(layers.Dropout(0.5))
  M.add(layers.Dense(10, activation='softmax'))
  opt = keras.optimizers.Adam(learning_rate=0.00015)
  M.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
  reduce_cb = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.32, patience=5, restore_best_weights=True)
  early_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, min_delta=1e-5)
  M.fit(x_train, y_train, batch_size=256, epochs=100, verbose=0, validation_data=(x_val, y_val),
    callbacks=[reduce_cb, early_cb])
  _, acc = M.evaluate(x_test, y_test, verbose=0)
  return acc

In [4]:
import time; start = time.time()
keras.utils.set_random_seed(seed=23); num_exp = 2; acc = np.zeros(num_exp)
for exp in range(num_exp):
    acc[exp] = run_exp3()
print(f'Precisión media: {acc.mean():.2%} Desviación estándar: {acc.std():.2%}')
print('Tiempo (hh:mm:ss):', time.strftime('%H:%M:%S', time.gmtime(time.time() - start)))

I0000 00:00:1733939366.321116  854554 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2616 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1733939368.223517  875960 service.cc:148] XLA service 0x7f074400b350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733939368.223548  875960 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
I0000 00:00:1733939368.356017  875960 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1733939370.567196  875960 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Precisión media: 75.89% Desviación estándar: 0.08%
Tiempo (hh:mm:ss): 00:05:08
